In [ ]:
# Cargar los datos y obtener el modelo
from data_and_model_construction.data import create_data_dict
from data_and_model_construction.common_model import create_model, solve_model
from plot_kind.costo_op_vs_disp import CostoOportunidad

data = create_data_dict()
mdl, production_vars, products = create_model(data)
solve_model(mdl, production_vars, products)


Model: produccion_pullover
 - number of variables: 5
   - binary=0, integer=0, continuous=5
 - number of constraints: 15
   - linear=15
 - parameters: defaults
 - objective: maximize
 - problem type is: LP
* Production model solved with objective: 550
* Total benefit=550
Production of A: 0
Production of B: 16.666666666666664
Production of B1: 13.333333333333332
Production of B2: 3.333333333333332
Production of C: 16.666666666666668


In [ ]:
# Usage of the iterate_over_rhs method, checking whether there exists demmin or not
constraint_nameX = 'DemandMin_B'  # The name of the constraint to analyze
product_name="B"

costo_op = CostoOportunidad(mdl, products, production_vars)
current_rhs_value, rhs_values, dual_values = costo_op.iterate(constraint_nameX, product_name)

Demanda mínima encontrada para el producto B.
[debug] (lower, upper): (-1e+20, 16.666666666666664)
---
- Adjusting RHS to: 16.676666666666666
* Production model solved with objective: 549.97
* Total benefit=549.97
Production of A: 0
Production of B: 16.676666666666666
Production of B1: 13.333333333333332
Production of B2: 3.3433333333333337
Production of C: 16.656666666666666


---
- Adjusting RHS to: 20.0
* Production model solved with objective: 540
* Total benefit=540
Production of A: 0
Production of B: 20.0
Production of B1: 13.333333333333332
Production of B2: 6.666666666666668
Production of C: 13.333333333333332
---
- Adjusting RHS to: 20.01
No solution found for RHS value: 20.01


In [ ]:
costo_op.plot(constraint_nameX, product_name, "[un/semana]", "[$/un]")

[debug] current_x_value: 10


<Figure size 640x480 with 1 Axes>

<Figure size 2000x1000 with 0 Axes>